# TradingClient example

## Prepare client

In [ ]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [ ]:
# import the packages
import osmosispy
from osmosispy import pytypes
from osmosispy.trading_client import BinanceTradingClient, TradeData
import time

don't forget to set your API key and secret to the environment variables
`BINANCE_API_KEY` and `BINANCE_API_SECRET` respectively. Use [this article](https://www.binance.com/en/support/faq/how-to-create-api-360002502072) to get api keys

```bash
export BINANCE_API_KEY=qwerty
export BINANCE_API_SECRET=asdfgh
```

In [ ]:
# Create a binance trading client
binance_client = BinanceTradingClient.from_env()

# # Alternatively, you can pass them to the client constructor.
# binance_client = BinanceTradingClient(
#     api_key="qwerty",
#     api_secret="asdfgh",
# )

## Get OSMO USDT trading info

In [ ]:
fee = binance_client.get_trading_fee("OSMOUSDT")
# fee object will contain the following fields:
# maker_commision: float
# taker_commision: float

print(f"Trading fee: {fee}")

In [ ]:
price_in_usdt = binance_client.get_price(symbol = "OSMOUSDT")

print(f"Current price in USDT:\n{price_in_usdt}")

## Compare OSMO prices on binance and Osmosis
Simple example of comparing OSMO prices on binance and Osmosis.

In [ ]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import (
    query_pb2 as gamm_query,
    query_pb2_grpc as gamm_query_grpc,
)
from osmosispy import Sdk, Network, utils

In [ ]:
# prepare the osmosis client
mnemonic_key = "satisfy adjust timber high purchase tuition stool faith fine install that you unaware feed domain license impose boss human eager hat rent enjoy dawn"

network = Network.mainnet()
trader = Sdk.authorize(key=mnemonic_key).with_network(network).with_config(pytypes.TxConfig())

stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)

# check the address
print(trader.address)

In [ ]:

stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
pool_raw_data = utils.deserialize(
    stub_gamm.Pool(
        # checking this pool https://app.osmosis.zone/pool/678
        gamm_query.QueryPoolRequest(pool_id=678)
    )
)

osmosis_pool = utils.deserialize_pool(pool_raw_data["pool"])

print(f"osmosis pool: {osmosis_pool}")

In [ ]:
def get_osmosis_trading_price(pool):
    price_data = utils.deserialize(
        stub_gamm.SpotPrice(
            gamm_query.QuerySpotPriceRequest(
                pool_id=pool["id"],
                base_asset_denom=pool["pool_assets"][0]["token"]["denom"],
                quote_asset_denom=pool["pool_assets"][1]["token"]["denom"],
            )
        )
    )

    return price_data["spot_price"]

# create trade callback
def trade_callback(trade: TradeData):
    print("===== OSMO USDT trades =====")
    print(f"binance price: {trade.price.amount}\nbinance quantity: {trade.quantity.amount}")
    
    osmosis_pool_price = get_osmosis_trading_price(osmosis_pool);
    print(f"osmosis price: {osmosis_pool_price}")

# start listening to trades
binance_client.listen_trades(cb=trade_callback, symbol="OSMOUSDT")

In [ ]:
# listen for 60 seconds
time.sleep(60)

In [ ]:
# stop listening to trades
binance_client.stop_listeners()